In [3]:
from CLIP import CLIPModel
import torch

In [10]:
checkpoint = torch.load('logs/shared_projector_shared_encoder/best.pt', map_location='cpu')
print(checkpoint.keys())


odict_keys(['image_encoder1.model.stages.0.0.convs.0.conv.weight', 'image_encoder1.model.stages.0.0.convs.0.conv.bias', 'image_encoder1.model.stages.0.0.convs.0.norm.weight', 'image_encoder1.model.stages.0.0.convs.0.norm.bias', 'image_encoder1.model.stages.0.0.convs.0.all_modules.0.weight', 'image_encoder1.model.stages.0.0.convs.0.all_modules.0.bias', 'image_encoder1.model.stages.0.0.convs.0.all_modules.1.weight', 'image_encoder1.model.stages.0.0.convs.0.all_modules.1.bias', 'image_encoder1.model.stages.0.0.convs.1.conv.weight', 'image_encoder1.model.stages.0.0.convs.1.conv.bias', 'image_encoder1.model.stages.0.0.convs.1.norm.weight', 'image_encoder1.model.stages.0.0.convs.1.norm.bias', 'image_encoder1.model.stages.0.0.convs.1.all_modules.0.weight', 'image_encoder1.model.stages.0.0.convs.1.all_modules.0.bias', 'image_encoder1.model.stages.0.0.convs.1.all_modules.1.weight', 'image_encoder1.model.stages.0.0.convs.1.all_modules.1.bias', 'image_encoder1.model.stages.1.0.convs.0.conv.weight

In [13]:
torch.eq(checkpoint['image_encoder1.model.stages.0.0.convs.0.conv.weight'], checkpoint['image_encoder2.model.stages.0.0.convs.0.conv.weight']).all()

tensor(True)

In [8]:
from nnunetv2.run.run_training import get_trainer_from_args
from nnunetv2.run.load_pretrained_weights import load_pretrained_weights
import config as CFG

trainer = get_trainer_from_args(
    dataset_name_or_id=CFG.nnUNet['dataset_name_or_id'],
    configuration=CFG.nnUNet['configuration'],
    fold=CFG.nnUNet['fold'],
    trainer_name=CFG.nnUNet['trainer_name'],
    plans_identifier=CFG.nnUNet['plans_identifier'],
    device=torch.device('cpu'))
trainer.initialize()

# trainer.load_checkpoint(CFG.nnUNet['checkpoint'])

Using device: cpu

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################



/home/gridsan/nchutisilp/.local/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [20]:
from collections import OrderedDict

encoder_checkpoint = OrderedDict()
for k, v in checkpoint.items():
    if 'image_encoder1.' in k:
        encoder_checkpoint[k.replace('image_encoder1.model.', '')] = v
    
# print(encoder_checkpoint)
load_success = trainer.network.encoder.load_state_dict(encoder_checkpoint)
print(load_success)
trainer.save_checkpoint('clip_pretrained_nnUNet.pt')

<All keys matched successfully>
